# Приложение для чтения книг (работа с SQL)

## Содержание: <a name="content"></a>
- [Загрузка библиотек и данных](#loading)
- [Исследуем таблицы](#explore_table)
- [Задачи:](#tasks)
  * [Сколько книг выпущенно после 1 января 2000 года;](#tasks_1)
  * [Сколько пользовательских обзоров и какова средняя оценка для каждой книги;](#tasks_2)
  * [Какое издательство издало наибольшее число книг толще 50 страниц;](#tasks_3)
  * [Какой автор  имеет самую высокую среднюю оценку книг, учитывая только книги с 50 и более пользовательскими оценками;](#tasks_4)
  * [Каково среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок.](#tasks_5)


## Загрузка библиотек и данных <a name="loading"></a>

[к содержанию](#content)

In [1]:
!pip install psycopg2-binary

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}, echo=False)

## Исследуем таблицы <a name="explore_table"></a>

[к содержанию](#content)

### Таблица books

In [4]:
query = '''
SELECT *
FROM books
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Таблица authors

In [5]:
query = '''
SELECT *
FROM authors
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Таблица publishers

In [6]:
query = '''
SELECT *
FROM publishers
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Таблица ratings

In [7]:
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Таблица reviews

In [8]:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Сделаем запросы под задачи <a name="tasks"></a>

[к содержанию](#content)

### Посчитаем количество книг, выпущенных после 1 января 2000 года: <a name="tasks_1"></a>

[к содержанию](#content)

In [9]:
query = '''
SELECT COUNT(title) AS count_books
FROM books
WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine)

,count_books
0,819


После 1 января 2000 года было выпущено 819 книг.

### Посчитаем количество пользовательских обзоров и среднюю оценку для каждой книги: <a name="tasks_2"></a>

[к содержанию](#content)

In [15]:
query = '''
SELECT books.title, COUNT(DISTINCT reviews.review_id) AS count_reviews, AVG(ratings.rating) AS avg_rating
FROM books
LEFT JOIN reviews ON books.book_id = reviews.book_id
LEFT JOIN ratings ON books.book_id = ratings.book_id
            
GROUP BY books.title
LIMIT 10;
'''
gg = pd.io.sql.read_sql(query, con = engine)
gg

,title,count_reviews,avg_rating
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
5,1st to Die (Women's Murder Club #1),4,3.500000
6,2nd Chance (Women's Murder Club #2),3,3.000000
7,4th of July (Women's Murder Club #4),3,4.000000
8,A Beautiful Mind,2,4.250000
9,A Bend in the Road,3,3.200000


### Определим издательство, которое издало наибольшее число книг толще 50 страниц: <a name="tasks_3"></a>

[к содержанию](#content)

In [16]:
query = '''
SELECT publishers.publisher, COUNT(books.book_id) AS count_books
FROM books
LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.book_id IN
    (SELECT books.book_id
     FROM books
     GROUP BY books.book_id
     HAVING books.num_pages > 50)
GROUP BY publishers.publisher
ORDER BY COUNT(books.book_id) DESC
LIMIT 1;

'''
pd.io.sql.read_sql(query, con = engine)

,publisher,count_books
0,Penguin Books,42


Издательство, которое издало наибольшее число книг толще 50 страниц - Penguin Books

### Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более пользовательскими оценками <a name="tasks_4"></a>

[к содержанию](#content)

In [17]:
query = '''
SELECT authors.author, AVG(ratings.rating)
FROM books
LEFT JOIN authors ON books.author_id = authors.author_id
LEFT JOIN ratings ON books.book_id = ratings.book_id
WHERE books.book_id IN

    (SELECT books.book_id
     FROM books
     LEFT JOIN ratings ON books.book_id = ratings.book_id
            
GROUP BY books.book_id
HAVING count(ratings.rating) > 49)

GROUP BY authors.author
ORDER BY AVG(ratings.rating) DESC
LIMIT 1;

'''
pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


Автор с самой высокой средней оценкой книг (учитывая только книги с 50 и более пользовательскими оценками) - Джоан Кэтлин Роулинг.

### Посчитаем среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок <a name="tasks_5"></a>

[к содержанию](#content)

In [18]:
query = '''
SELECT AVG(sub.count)
FROM 
    (SELECT count(*)
     FROM reviews
     WHERE username IN
         (SELECT username
          FROM ratings
          GROUP BY username
          HAVING count(rating) > 50)
    GROUP BY username) AS sub;

'''
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333


Cреднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок, составляет 24